# ME-обращение с ошибками

In [1]:
import sys
sys.path.append('..')

In [2]:
import time

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

from tqdm.auto import tqdm

In [3]:
import torch

from inverse_problem import me_model
from inverse_problem import PregenSpectrumDataset
from inverse_problem.nn_inversion.posthoc import open_param_file
from inverse_problem.nn_inversion.transforms import inverse_transformation

from inverse_problem.nn_inversion.model_pi_mlp_conv import PIMLPConvNet
from inverse_problem.nn_inversion import conv1d_transform_rescale


np.set_printoptions(suppress=True)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
path_to_refer = '../data/hinode_source/20140510_105533.fits'

ref, names = open_param_file(path_to_refer, normalize=False, print_params=False)
parameter_base = ref.reshape(-1, 11)

In [5]:
def me_lm_prepare(line_arg, *params):
    line_vec = (6302.5, 2.5, 1)
    param_vec = np.array([*params])
    lines = me_model(param_vec, line_arg, line_vec, with_ff=True, with_noise=False, norm=False)
    return lines.reshape(1, -1, order='F').flatten()

In [6]:
def get_lines(param_vec):
    line_vec = (6302.5, 2.5, 1)
    line_arg = 1000 * (np.linspace(6302.0692255, 6303.2544205, 56) - line_vec[0])
    
    real_lines = me_model(param_vec, line_arg, line_vec, with_ff=True, with_noise=True, norm=False)
    
    return real_lines.reshape(1, -1, order='F').flatten()


def predict_one_pixel(param_vec, start_p=None):
    start = time.time()

    lines = get_lines(param_vec)
    
    if start_p is None:
        start_p =  [530, 91, 89, 33, 0.31, 12, 27083, 19567, 0.04, 0.5, 0.36]
    
    max_ = [5000, 180, 180, 90, 1.5, 100, 38603, 60464, 10, 1, 10]
    min_ = [0, 0, 0, 20, 0, 0.01, 0, 0, -10, 0, -10]

    line_arg = 1000 * (np.linspace(6302.0692255, 6303.2544205, 56) - 6302.5)
    
    try:
        out = curve_fit(me_lm_prepare, xdata=line_arg, ydata=lines, bounds=(min_, max_), p0=start_p)
    except:
        out = (np.array(start_p), np.zeros((11, 11)))
    
    elapsed = (time.time() - start)
   
    pcov = out[1]
    perr = np.sqrt(np.diag(pcov))
    
    data = np.concatenate([param_vec.reshape(11, -1), out[0].reshape(11, -1), perr.reshape(11, -1)], axis=1)
    
    return data, elapsed

## Параметры

In [7]:
batch_size = 4096

## Модельные спектры

In [8]:
def me_comp(batch_size=1024, mode='standart', start_p=None):
    parameter_base_length = parameter_base.shape[0]
    n_batches = parameter_base_length // batch_size

    batch_num = 0
    
    res_ar = np.zeros((batch_size, 11, 3))
    time_ar = np.zeros((batch_size, 1))
    
    pbar1 = tqdm(total=n_batches, position=0, desc='Batches')
    pbar2 = tqdm(total=batch_size, position=1, desc='Batch', leave=False)
    
    if mode == 'standart':
        save_names = ['../me-inv-with-sigmas/params_comp_',
                      '../me-inv-with-sigmas/time_']
    elif mode == 'NN':
        save_names = ['../me-inv-with-sigmas/params_comp_NN_',
                      '../me-inv-with-sigmas/time_NN_']
        assert start_p is not None
    else:
        raise NotImplemented

    
    for i in range(parameter_base_length):
        res_ar[i - batch_num * batch_size, ...], time_ar[i - batch_num * batch_size] = predict_one_pixel(parameter_base[i],
                                                                                                         start_p=None if start_p is None else start_p[i]) 
        
        pbar2.update()
        
        if (i == batch_size * (batch_num + 1) - 1 and i > 0) or i == parameter_base_length - 1:
            np.save(save_names[0] + f'{batch_size}_{batch_num}', res_ar)
            np.save(save_names[1] + f'{batch_size}_{batch_num}', time_ar)
            
            batch_num += 1
            
            pbar1.update(1)
            
            pbar2.close()
            pbar2 = tqdm(total=batch_size, position=1, desc='Batch', leave=False)
            
            if parameter_base_length - i > batch_size:
                res_ar = np.zeros((batch_size, 11, 3))
                time_ar = np.zeros((batch_size, 1))
            else:
                size_0 = parameter_base_length - i - 1
                res_ar = np.zeros((size_0, 11, 3))
                time_ar = np.zeros((size_0, 1))        

### Старт из среднего

---

In [9]:
idx_0 = 12398
test_params_0 = parameter_base[idx_0]
print(idx_0, '\n', *test_params_0.round(2), sep='')

12398
152.1289.0678.0420.180.02100.05955.569807.231.060.190.1


In [10]:
out, tm = predict_one_pixel(test_params_0)

---

In [ ]:
me_comp(batch_size=batch_size)

Batches:   0%|          | 0/109 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

...

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

Batch:   0%|          | 0/4096 [00:00<?, ?it/s]

In [27]:
batch_num = 0

results = np.load(f'../me-inv-with-sigmas/params_comp_{batch_size}_{batch_num}.npy')
elapsed_t = np.load(f'../me-inv-with-sigmas/time_{batch_size}_{batch_num}.npy')

In [28]:
par_ind = 0

df1 = pd.DataFrame(results[par_ind], columns = ['target', 'predicted', 'sigmas'])

print('Затраченное на вычисление время:', np.round(elapsed_t[par_ind][0], 2), 'секунд')
df1.round(2)

Затраченное на вычисление время: 0.1 секунд


,target,predicted,sigmas
0,1400.28,1425.16,15.72
1,32.97,32.89,0.75
2,177.85,178.82,1.27
3,38.84,41.51,1.14
4,0.27,0.23,0.02
5,2.50,1.80,0.23
6,12836.86,10302.78,1275.28
7,18536.04,21070.43,1273.46
8,0.14,0.13,0.02
9,0.56,0.55,0.01


## Старт из нейропрогноза

### Инициализация модели

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device.type)

In [ ]:
model = PIMLPConvNet(n_blocks=6, in_dim=(4, 64, 64, 128, 128, 256), out_dim=(64, 64, 128, 128, 256, 256),
                     kernel_size=(3, 3, 3, 3, 3, 3), padding=(1, 1, 1, 1, 1, 1), activation='elu', dropout=0.05,
                     batch_norm=True, pool=(None, 'max', None, 'max', None, None), hidden_dims=(100, 100),
                     bottom_output=100, number_readout_layers=2, top_output=11)

In [ ]:
checkpoint_path = '../models/wl_model_2022-02-27_03-11/wl_model_ep4.pt'
model.load_state_dict(torch.load(checkpoint_path, map_location=device)['model_state_dict'])

### Предсказание

In [ ]:
factors, cont_scale = [1, 1000, 1000, 1000], 40000
angle_transformation, logB = True, True

transform_name = "conv1d_transform_rescale"

transform = conv1d_transform_rescale(factors=factors, 
                                     angle_transformation=angle_transformation,
                                     cont_scale=cont_scale, logB=logB)

In [ ]:
test_dataset = PregenSpectrumDataset(data_arr=parameter_base,
                                     transform=transform)

In [ ]:
inputs = test_dataset.samples
sim_x = [inputs['X'][0].to(device), inputs['X'][1].to(device)]
y = inputs['Y'].to(device)

In [ ]:
model.eval()
with torch.no_grad():
    predicted = model(sim_x)
        
predicted = predicted.numpy()
predicted[:, 1:3] = np.clip(predicted[:, 1:3], 0, 1)    
        
model_pred = inverse_transformation(predicted,
                                    inv_logB=logB,
                                    inv_angle_transformation=angle_transformation)

In [ ]:
me_comp(batch_size=batch_size, mode='NN', start_p=model_pred)

In [30]:
batch_num = 0

results_m = np.load(f'../me-inv-with-sigmas/params_comp_NN_{batch_size}_{batch_num}.npy')
elapsed_m_t = np.load(f'../me-inv-with-sigmas/time_NN_{batch_size}_{batch_num}.npy')

In [31]:
par_ind = 0

df2 = pd.DataFrame(results_m[par_ind], columns = ['target', 'predicted', 'sigmas'])

print('Затраченное на вычисление время:', np.round(elapsed_m_t[par_ind][0], 2), 'секунд')
df2.round(2)

Затраченное на вычисление время: 0.03 секунд


,target,predicted,sigmas
0,1400.28,1389.35,17.40
1,32.97,32.11,0.78
2,177.85,176.84,1.36
3,38.84,36.58,1.03
4,0.27,0.30,0.02
5,2.50,3.04,0.38
6,12836.86,13818.44,847.09
7,18536.04,17577.29,845.30
8,0.14,0.10,0.02
9,0.56,0.55,0.01


## Analysis

### Standart

In [105]:
f_names = ['../me-inv-with-sigmas/params_comp_4096_' + str(i) + '.npy' for i in range(109)]
f_names_time = ['../me-inv-with-sigmas/time_4096_' + str(i) + '.npy' for i in range(109)]

In [97]:
st_res = np.empty((len(parameter_base), 11, 3))
st_time = np.empty((len(parameter_base), 1))

In [98]:
for index, f_name in enumerate(f_names[:-1]):
    st_res[index * batch_size:(index + 1) * batch_size, ...] = np.load(f_name)
  
t_ar = np.load(f_names[-1])
st_res[-t_ar.shape[0]:, ...] = t_ar

In [107]:
for index, f_name in enumerate(f_names_time[:-1]):
    st_time[index * batch_size:(index + 1) * batch_size, ...] = np.load(f_name)
  
t_ar = np.load(f_names_time[-1])
st_time[-t_ar.shape[0]:, ...] = t_ar

In [151]:
st_time.sum() / 3600

43.28148776157496

In [149]:
print(f'Время обсчета одного снимка: {st_time.sum() / 3600} часов')

Время обсчета одного снимка: 43.28148776157496 часов


In [125]:
np.save('../me-inv-with-sigmas/params_comp', st_res)
np.save('../me-inv-with-sigmas/time', st_time)

### NN

In [106]:
f_names_NN = ['../me-inv-with-sigmas/params_comp_NN_4096_' + str(i) + '.npy' for i in range(109)]
f_names_NN_time = ['../me-inv-with-sigmas/time_NN_4096_' + str(i) + '.npy' for i in range(109)]

In [101]:
nn_res = np.empty((len(parameter_base), 11, 3))
nn_time = np.empty((len(parameter_base), 1))

In [102]:
for index, f_name in enumerate(f_names_NN[:-1]):
    nn_res[index * batch_size:(index + 1) * batch_size, ...] = np.load(f_name)
  
t_ar = np.load(f_names[-1])
nn_res[-t_ar.shape[0]:, ...] = t_ar

In [121]:
for index, f_name in enumerate(f_names_NN_time[:-1]):
    nn_time[index * batch_size:(index + 1) * batch_size, ...] = np.load(f_name)
  
t_ar = np.load(f_names_NN_time[-1])
nn_time[-t_ar.shape[0]:, ...] = t_ar

In [150]:
print(f'Время обсчета одного снимка: {nn_time.sum() / 3600} часов')

Время обсчета одного снимка: 20.245917834603553 часов


In [124]:
# np.save('../me-inv-with-sigmas/params_comp_NN', nn_res)
# np.save('../me-inv-with-sigmas/time_NN', nn_time)

In [157]:
st_res[0]

array([[ 1400.28259277,  1425.16210647,    15.71538479],
       [   32.97245789,    32.89359475,     0.7511327 ],
       [  177.85369873,   178.81658026,     1.27165123],
       [   38.83941269,    41.51282274,     1.14378937],
       [    0.26864216,     0.22861971,     0.01849041],
       [    2.50168061,     1.79861568,     0.23226476],
       [12836.86230469, 10302.7753237 ,  1275.27970767],
       [18536.04101562, 21070.43349022,  1273.45608733],
       [    0.13613635,     0.13089313,     0.01524388],
       [    0.5580886 ,     0.5458855 ,     0.00754621],
       [    0.50357759,     0.53145481,     0.018767  ]])

In [156]:
nn_res[0]

array([[ 1400.28259277,  1389.34847162,    17.40158867],
       [   32.97245789,    32.11031145,     0.78075615],
       [  177.85369873,   176.83924402,     1.36348685],
       [   38.83941269,    36.58407116,     1.03391411],
       [    0.26864216,     0.29525058,     0.01979214],
       [    2.50168061,     3.04019025,     0.3782804 ],
       [12836.86230469, 13818.438106  ,   847.09327374],
       [18536.04101562, 17577.29366666,   845.29586675],
       [    0.13613635,     0.09513147,     0.01578066],
       [    0.5580886 ,     0.55322678,     0.0088986 ],
       [    0.50357759,     0.52684336,     0.02128912]])

In [136]:
st_res[:, :, 2].mean() / nn_res[:, :, 2].mean()

2.7668264066918016